# Populate Azure AI Search Index

In [1]:
import os
import dotenv
import sys

dotenv.load_dotenv(".env")
sys.path.append(os.path.join(os.getcwd(), "..", "src"))

### Approach 1: Pull-based

The pull model uses indexers connecting to a supported data source, automatically uploading the data into your index. This is the recommended approach for data sources that are frequently updated.

In [2]:
from search.utilities import SearchClient

# Create search client
search_client = SearchClient(
    search_endpoint=os.environ["AZURE_AI_SEARCH_ENDPOINT"],
)

In [3]:
# Generate list of variables to be used in templates
template_variables = {
    key: value for key, value in os.environ.items() if key.startswith(("AZURE"))
}

# Define template paths
base_path = os.path.join(os.getcwd(), "..", "src", "search", "templates")
datasource_template_path = os.path.join(base_path, "product-info", "datasource.json")
index_template_path = os.path.join(base_path, "product-info", "index.json")
skillset_template_path = os.path.join(base_path, "product-info", "skillset.json")
indexer_template_path = os.path.join(base_path, "product-info", "indexer.json")

# List of search assets
assets = [
    {
        "type": "indexes",
        "name": os.environ["AZURE_AI_SEARCH_INDEX_NAME"],
        "template_path": index_template_path,
        "template_variables": template_variables,
    },
    {
        "type": "datasources",
        "name": os.environ["AZURE_AI_SEARCH_DATASOURCE_NAME"],
        "template_path": datasource_template_path,
        "template_variables": template_variables,
    },
    {
        "type": "skillsets",
        "name": os.environ["AZURE_AI_SEARCH_SKILLSET_NAME"],
        "template_path": skillset_template_path,
        "template_variables": template_variables,
    },
    {
        "type": "indexers",
        "name": os.environ["AZURE_AI_SEARCH_INDEXER_NAME"],
        "template_path": indexer_template_path,
        "template_variables": template_variables,
    },
]

# Load search asset templates
search_client.load_search_management_asset_templates(assets)

In [4]:
# Create the index
index_response = search_client.create_search_management_asset(asset_type="indexes")

# Create the data source
datasource_response = search_client.create_search_management_asset(asset_type="datasources")

# Create skillset to enhance the indexer
skillset_response = search_client.create_search_management_asset(asset_type="skillsets")

# Create the indexer
indexer_response = search_client.create_search_management_asset(asset_type="indexers")

In [5]:
# Run the indexer
indexer_run_response = search_client.run_indexer()

In [9]:
# Run the indexer with reset
indexer_run_reset_response = search_client.run_indexer(reset_flag=True)